## The purpose of this notebook is to caculate the descriptors of a group of compounds
#### The input is a csv file containing the SMILES strings of the target compounds
#### Finally the dataframe of descriptors for each compund is exported as a csv file - qsar object.
#### The process is done in batches of 10.000.000 compounds. It takes 11 runs to complete the 110 million compounds in PubChem

import pubchempy as pcp
import pandas as pd
import numpy as np
import time

from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem import PandasTools

from mordred import Calculator, descriptors
from mordred import  ABCIndex

#### The path down there should be design to store a copy in the user's pc. in that case "../DirectChlor/data/smiles_20201120_20000000_30000000.csv" should work right? 

In [125]:
smiles_df = pd.read_csv("excel_py/smiles_20201120_20000000_30000000.csv")#6200000
# smiles_df1 = smiles_df.iloc[0:5000000]
# smiles_df2 = smiles_df.iloc[5000000:10000000]

In [126]:
features_list_TCAA_rf = ["SsOH",
                         "C2SP2",
                        "SlogP_VSA11",
                        "SMR_VSA7",
                        "PEOE_VSA1",
                        "EState_VSA7",
                        "ATSC5are",
                        "nBondsKD",
                        "ATS1i",
                        "nBondsM"]

features_list_DCAA_rf = ["AATSC0are",
                         "ATSC3dv",
                         "SlogP_VSA11",
                         "ATS4m",
                         "ATS4Z",
                         "Xch-7d",
                         "PEOE_VSA1",
                         "SaasC",
                         "piPC4",
                         "VSA_EState9"]
features_list_TCAA_DCAA_rf = ["SsOH",
                         "C2SP2",
                        "SlogP_VSA11",
                        "SMR_VSA7",
                        "PEOE_VSA1",
                        "EState_VSA7",
                        "ATSC5are",
                        "nBondsKD",
                        "ATS1i",
                        "nBondsM",
                        "AATSC0are",
                        "ATSC3dv",
                        "ATS4m",
                        "ATS4Z",
                        "Xch-7d",
                        "SaasC",
                        "piPC4",
                        "VSA_EState9" ] 

calc_dummy = Calculator(descriptors, ignore_3D=False)
my_desc_names_TCAA = features_list_TCAA_rf
my_desc_names_DCAA = features_list_DCAA_rf
my_desc_names_TCAA_DCAA = features_list_TCAA_DCAA_rf                        
my_descs = []
for i, desc in enumerate(calc_dummy.descriptors):
    if desc.__str__()  in my_desc_names_TCAA:
       my_descs.append(desc)

calc_TCAA = Calculator(my_descs, ignore_3D=False)
my_descs = []
for i, desc in enumerate(calc_dummy.descriptors):
    if desc.__str__()  in my_desc_names_DCAA:
       my_descs.append(desc)

calc_DCAA = Calculator(my_descs, ignore_3D=False)
my_descs = []
for i, desc in enumerate(calc_dummy.descriptors):
    if desc.__str__()  in my_desc_names_TCAA_DCAA:
       my_descs.append(desc)

calc_TCAA_DCAA = Calculator(my_descs, ignore_3D=False)

In [127]:
calc = calc_TCAA_DCAA

In [128]:
start = time.time()
smiles_df_List = []
smilesdivisionNumber = 1000000
smiles_df_result_List = []
tem_df = smiles_df
smiles_number = len(tem_df)
while smiles_number >= smilesdivisionNumber:
    smiles_df_List.append(tem_df.iloc[:smilesdivisionNumber])
    tem_df = tem_df.iloc[smilesdivisionNumber:]
    smiles_number = smiles_number-smilesdivisionNumber
    
if smiles_number > 0:
    smiles_df_List.append(tem_df)
print("smiles ok")
elapsed_time = time.time() - start
print ("elapsed_time:{0}".format(elapsed_time) + "[sec]")

j = 1

smiles ok
elapsed_time:0.8109691143035889[sec]


In [129]:
for i in  smiles_df_List:
    k = 1
    tem_time = time.time()
    now_df = i
    now_df_number = len(now_df)
    divisionList = []
    resultList = []
    divisionNumber = 100000
    while now_df_number >= divisionNumber:
        divisionList.append(now_df.iloc[:divisionNumber])
        now_df = now_df.iloc[divisionNumber:]
        now_df_number = now_df_number - divisionNumber
    if  now_df_number > 0:
        divisionList.append(now_df)
    for i in divisionList:
        div_df = i
        PandasTools.AddMoleculeColumnToFrame(div_df, smilesCol='CanonicalSMILES', includeFingerprints=False)
        resultList.append(div_df)
    now_df = pd.concat(resultList, axis=0)
    now_df = now_df.dropna(subset=['ROMol'])
    now_df = now_df.drop('CanonicalSMILES', axis=1)
    print(str(j)+"回目のROMol ok")
    elapsed_time = time.time() - tem_time
    tem_time = time.time()
    print ("elapsed_time:{0}".format(elapsed_time) + "[sec]")

    divisionList2 = []
    resultList2 = []
    divisionNumber2 = 10000
    now_df_number = len(now_df)
    while now_df_number >= divisionNumber2:
        divisionList2.append(now_df.iloc[:divisionNumber2])
        now_df = now_df.iloc[divisionNumber2:]
        now_df_number = now_df_number - divisionNumber2

    if now_df_number > 0:
        divisionList2.append(now_df)

    for i in divisionList2:
        div_df = i
        div_mol_list = div_df['ROMol']
        print(k)
        k = k + 1
        tem_qsar_preview = pd.DataFrame(calc.pandas(div_mol_list, nproc=4).fill_missing())
        div_df = pd.concat([div_df.drop('ROMol', axis=1), tem_qsar_preview], axis=1)
        resultList2.append(div_df)
    tem_qsar_df = pd.concat(resultList2, axis=0)
    smiles_df_result_List.append(tem_qsar_df)
    print(str(j)+"回目のDescriptor ok")
    elapsed_time = time.time() - tem_time
    tem_time = time.time()
    print ("elapsed_time:{0}".format(elapsed_time) + "[sec]")
    j = j + 1
qsar_df = pd.concat(smiles_df_result_List, axis=0)
elapsed_time = time.time() - start
print ("elapsed_time:{0}".format(elapsed_time) + "[sec]")

RDKit WARNING: [01:38:35] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [01:38:35] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [01:38:35] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [01:38:35] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [01:38:35] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [01:38:36] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [01:38:36] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [01:38:36] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [01:38:37] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [01:38:37] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [01:38:37] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [01:38:37] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [01:38:37] WARNING: not r

1回目のROMol ok
elapsed_time:216.72467422485352[sec]
1


100%|██████████| 10000/10000 [00:50<00:00, 199.86it/s]


2


100%|██████████| 10000/10000 [00:48<00:00, 205.52it/s]


...

99


100%|██████████| 10000/10000 [00:59<00:00, 167.45it/s]


100


100%|██████████| 10000/10000 [00:54<00:00, 184.64it/s]


10回目のDescriptor ok
elapsed_time:5312.724675178528[sec]
elapsed_time:66163.84141921997[sec]


In [123]:
try_df = smiles_df_List[2].iloc[157508:157509]

In [124]:
#CID 12157941エラー

,CID,CanonicalSMILES
2157508,12157941,[B-](C1=C(C(=C(C(=C1F)F)F)F)F)(C2=C(C(=C(C(=C2...


In [119]:
PandasTools.AddMoleculeColumnToFrame(try_df, smilesCol='CanonicalSMILES', includeFingerprints=False)

In [120]:
try_df

RDKit ERROR: [22:57:55] Can't kekulize mol.  Unkekulized atoms: 50 51 53
RDKit ERROR: 
RDKit ERROR: [22:57:55] Can't kekulize mol.  Unkekulized atoms: 50 51 53
RDKit ERROR: 


KekulizeException: Can't kekulize mol.  Unkekulized atoms: 50 51 53


RDKit ERROR: [22:57:55] Can't kekulize mol.  Unkekulized atoms: 50 51 53
RDKit ERROR: 
RDKit ERROR: [22:57:55] Can't kekulize mol.  Unkekulized atoms: 50 51 53
RDKit ERROR: 
RDKit ERROR: [22:57:55] Can't kekulize mol.  Unkekulized atoms: 50 51 53
RDKit ERROR: 
RDKit ERROR: [22:57:55] Can't kekulize mol.  Unkekulized atoms: 50 51 53
RDKit ERROR: 


KekulizeException: Can't kekulize mol.  Unkekulized atoms: 50 51 53


In [87]:
try_mol_list = try_df['ROMol']

In [88]:
try_qsar_preview = pd.DataFrame(calc.pandas(try_mol_list, nproc=4).fill_missing())

  0%|          | 0/5 [00:00<?, ?it/s]RDKit ERROR: [22:55:57] Can't kekulize mol.  Unkekulized atoms: 50 51 53
RDKit ERROR: 
  0%|          | 0/5 [00:00<?, ?it/s]


KekulizeException: Can't kekulize mol.  Unkekulized atoms: 50 51 53


In [131]:
qsar_df.head()

,CID,ATS4Z,ATS4m,ATS1i,ATSC3dv,ATSC5are,AATSC0are,nBondsM,nBondsKD,C2SP2,Xch-7d,SaasC,SsOH,PEOE_VSA1,SMR_VSA7,SlogP_VSA11,EState_VSA7,VSA_EState9,piPC4
0,20001040,3073.0,10944.203239,10512.5,-73.588695,6.45482,0.182415,11,8,5,0.142259,0.563930,28.785723,42.737283,29.82892,5.749512,6.255769,0.0,4.756818
1,20001041,3079.0,10983.899815,10292.1,-75.395617,7.1182,0.193303,11,8,5,0.142259,0.541152,37.548150,42.110143,29.82892,5.749512,11.572558,0.0,4.756818
2,20001042,3091.0,11034.770692,9930.94,-40.127595,3.6122,0.169453,10,7,5,0.142259,0.630452,28.648252,37.003615,29.82892,5.749512,6.255769,0.0,4.703884
3,20001043,3095.0,10947.051515,10725.1,-79.021557,3.64431,0.188008,11,8,5,0.142259,0.558513,37.698216,42.110143,29.82892,5.749512,6.255769,0.0,4.748188
4,20001044,3095.0,10933.248947,10945.5,-75.919067,3.83148,0.177457,11,8,5,0.142259,0.578159,28.976368,42.737283,29.82892,5.749512,6.255769,0.0,4.748188


In [132]:
qsar_df.to_csv("excel_py/qsar_20201120_20000000_30000000.csv",header=True,index=False)